In [4]:
# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
import pickle
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

#カラム内の文字数。デフォルトは50
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#
pd.options.display.float_format = '{:,.5f}'.format

%matplotlib inline

In [11]:

import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold

from sklearn import metrics
import json

import warnings
warnings.filterwarnings("ignore")

sys.path.append('..')
from lib.line_notif import send_message
from lib.utils import reduce_mem_usage, current_time, unpickle, to_pickle
from lib.utils import one_hot_encoder, apply_agg, multi_combine_categorical_feature
from lib.utils import import_data, get_split_indexer 

In [7]:
DATA_VERSION = "v001"
save_path = Path(f"../processed/{DATA_VERSION}")
save_path.mkdir(parents=True, exist_ok=True)

In [8]:
train = unpickle(save_path/"train_002.df.pkl", )
test  = unpickle(save_path/"test_002.df.pkl", )
y = train["scalar_coupling_constant"]
train.drop("scalar_coupling_constant", axis=1, inplace=True)
train.shape, test.shape, y.shape

((4658147, 450), (2505542, 450), (4658147,))

In [16]:
for f in ["atom_0"]:
    lbl = LabelEncoder()
    lbl.fit(list(train[f].values) + list(test[f].values))
    train[f] = lbl.transform(list(train[f].values))
    test[f] = lbl.transform(list(test[f].values))

In [18]:
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4658147 entries, 0 to 4658146
Data columns (total 450 columns):
id                                        int64
molecule_name                             object
atom_index_0                              int64
atom_index_1                              int64
type                                      int64
atom_0                                    int64
x_0                                       float64
y_0                                       float64
z_0                                       float64
atom_1                                    int64
x_1                                       float64
y_1                                       float64
z_1                                       float64
dist                                      float64
dist_x                                    float64
dist_y                                    float64
dist_z                                    float64
type_0                                    int64
m

In [21]:
train.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,dist_x,dist_y,dist_z,type_0,molecule_couples,molecule_dist_mean,molecule_dist_min,molecule_dist_max,molecule_dist_std,atom_0_couples_count,atom_1_couples_count,molecule_atom_index_0_count,molecule_atom_index_1_count,molecule_type_count,molecule_atom_1_count,molecule_type_0_count,molecule_atom_index_0_x_1_mean,molecule_atom_index_0_x_1_mean_diff,molecule_atom_index_0_x_1_mean_div,molecule_atom_index_0_x_1_max,molecule_atom_index_0_x_1_max_diff,molecule_atom_index_0_x_1_max_div,molecule_atom_index_0_x_1_std,molecule_atom_index_0_x_1_std_diff,molecule_atom_index_0_x_1_std_div,molecule_atom_index_0_x_1_min,molecule_atom_index_0_x_1_min_diff,molecule_atom_index_0_x_1_min_div,molecule_atom_index_0_y_1_mean,molecule_atom_index_0_y_1_mean_diff,molecule_atom_index_0_y_1_mean_div,molecule_atom_index_0_y_1_max,molecule_atom_index_0_y_1_max_diff,molecule_atom_index_0_y_1_max_div,molecule_atom_index_0_y_1_std,molecule_atom_index_0_y_1_std_diff,molecule_atom_index_0_y_1_std_div,molecule_atom_index_0_y_1_min,molecule_atom_index_0_y_1_min_diff,molecule_atom_index_0_y_1_min_div,molecule_atom_index_0_z_1_mean,molecule_atom_index_0_z_1_mean_diff,molecule_atom_index_0_z_1_mean_div,molecule_atom_index_0_z_1_max,molecule_atom_index_0_z_1_max_diff,molecule_atom_index_0_z_1_max_div,molecule_atom_index_0_z_1_std,molecule_atom_index_0_z_1_std_diff,molecule_atom_index_0_z_1_std_div,molecule_atom_index_0_z_1_min,molecule_atom_index_0_z_1_min_diff,molecule_atom_index_0_z_1_min_div,molecule_atom_index_0_dist_mean,molecule_atom_index_0_dist_mean_diff,molecule_atom_index_0_dist_mean_div,molecule_atom_index_0_dist_max,molecule_atom_index_0_dist_max_diff,molecule_atom_index_0_dist_max_div,molecule_atom_index_0_dist_std,molecule_atom_index_0_dist_std_diff,molecule_atom_index_0_dist_std_div,molecule_atom_index_0_dist_min,molecule_atom_index_0_dist_min_diff,molecule_atom_index_0_dist_min_div,molecule_atom_index_0_dist_x_mean,molecule_atom_index_0_dist_x_mean_diff,molecule_atom_index_0_dist_x_mean_div,molecule_atom_index_0_dist_x_max,molecule_atom_index_0_dist_x_max_diff,molecule_atom_index_0_dist_x_max_div,molecule_atom_index_0_dist_x_std,molecule_atom_index_0_dist_x_std_diff,molecule_atom_index_0_dist_x_std_div,molecule_atom_index_0_dist_x_min,molecule_atom_index_0_dist_x_min_diff,molecule_atom_index_0_dist_x_min_div,molecule_atom_index_0_dist_y_mean,molecule_atom_index_0_dist_y_mean_diff,molecule_atom_index_0_dist_y_mean_div,molecule_atom_index_0_dist_y_max,molecule_atom_index_0_dist_y_max_diff,molecule_atom_index_0_dist_y_max_div,molecule_atom_index_0_dist_y_std,molecule_atom_index_0_dist_y_std_diff,molecule_atom_index_0_dist_y_std_div,molecule_atom_index_0_dist_y_min,molecule_atom_index_0_dist_y_min_diff,molecule_atom_index_0_dist_y_min_div,molecule_atom_index_0_dist_z_mean,molecule_atom_index_0_dist_z_mean_diff,molecule_atom_index_0_dist_z_mean_div,molecule_atom_index_0_dist_z_max,molecule_atom_index_0_dist_z_max_diff,molecule_atom_index_0_dist_z_max_div,molecule_atom_index_0_dist_z_std,molecule_atom_index_0_dist_z_std_diff,molecule_atom_index_0_dist_z_std_div,molecule_atom_index_0_dist_z_min,molecule_atom_index_0_dist_z_min_diff,molecule_atom_index_0_dist_z_min_div,molecule_atom_index_1_x_1_mean,molecule_atom_index_1_x_1_mean_diff,molecule_atom_index_1_x_1_mean_div,molecule_atom_index_1_x_1_max,molecule_atom_index_1_x_1_max_diff,molecule_atom_index_1_x_1_max_div,molecule_atom_index_1_x_1_std,molecule_atom_index_1_x_1_std_diff,molecule_atom_index_1_x_1_std_div,molecule_atom_index_1_x_1_min,molecule_atom_index_1_x_1_min_diff,molecule_atom_index_1_x_1_min_div,molecule_atom_index_1_y_1_mean,molecule_atom_index_1_y_1_mean_diff,molecule_atom_index_1_y_1_mean_div,molecule_atom_index_1_y_1_max,molecule_atom_index_1_y_1_max_diff,molecule_atom_index_1_y_1_max_div,molecule_atom_index_1_y_1_std,molecule_atom_index_1_y_1_std_diff,molecule_atom_index_1_y_1_std_div,molecule_atom_in

In [22]:
train.drop("molecule_name", axis=1, inplace=True)
test.drop("molecule_name", axis=1, inplace=True)

In [23]:
categorical = ['atom_index_0', 'atom_index_1', 'atom_1', 'atom_0', 'type_0', 'type']

In [24]:
lgbm_params = {
    "boosting_type": "gbdt",
    'objective': 'regression',
    "metric": 'mae',
    # 'n_estimator': 10000,
    'n_jobs': -1,
    "seed": 71,
    "verbosity": -1,
    
    'learning_rate': 0.1,
    
    'max_depth': 9,
    'num_leaves': 128,
    
    "subsample_freq": 1,
    "subsample": 0.8,
    'colsample_bytree': 0.8,
    
    'min_child_samples': 100,
    'reg_alpha': 0.1,
    'reg_lambda': 0.3,
}

lgb_train = lgb.Dataset(train, y)

# Training settings
FOLD_NUM = 5
fold_seed = 71
folds = KFold(n_splits=FOLD_NUM, shuffle=True, random_state=fold_seed)

callbacks = [
    ModelExtractionCallback()
]

# Fitting
ret = lgb.cv(params=lgbm_params,
               train_set=lgb_train,
               categorical_feature=categorical,
               folds=folds,
               num_boost_round=10000,
               verbose_eval = 300,
               early_stopping_rounds=200,
               callbacks=callbacks,
               )
df_ret = pd.DataFrame(ret)
df_ret

MemoryError: 

In [ ]:
# Retrieving booster and training information.
proxy = extraction_cb.boosters_proxy
boosters = extraction_cb.raw_boosters
best_iteration = extraction_cb.best_iteration

In [ ]:
# Create oof prediction result
fold_iter = folds.split(train, y_train)
oof_preds = np.zeros_like(y_train_ohe)
for n_fold, ((trn_idx, val_idx), booster) in enumerate(zip(fold_iter, boosters)):
    print(val_idx)
    valid = train[val_idx]
    oof_preds[val_idx] = booster.predict(valid, num_iteration=best_iteration)
print(f"mae on oof preds: {mean_absolute_error(y_train, oof_preds)}")

In [ ]:
# Averaging prediction result for test data.
y_pred_proba_list = proxy.predict(test, num_iteration=best_iteration)
y_pred_proba_avg = np.array(y_pred_proba_list).mean(axis=0)
mae = mean_absolute_error(y_test, y_pred_proba_avg)
print('Averaging mae:', mae)

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['scalar_coupling_constant'] = y_pred_proba_avg
sub.to_csv('submission.csv', index=False)
sub.head()

# sandbox

In [ ]:

class LGB_cv(object):
    def __init__(self, num_boost_round=100):
        # set index of target features
        self.categorical_features = []
        self.callback = None
        self.num_boost_round = num_boost_round
        
    def get_model_name(self):
        return "lgb"
    
    def __str__(self):
        return self.get_model_name()
    
    def set_callback(self, callback):
        self.callback = callback
    
    def set_params(self, params):
        self.params = params
    
    def print_params(self):
        print(self.clf)
        clf_print = str(self.clf)
        return clf_print
    
    def set_train_data(self, X_train, y_train, X_valid, y_valid):
        self.X_train = X_train
        self.y_train = y_train
        self.X_valid = X_valid
        self.y_valid = y_valid
        
        self.lgb_train = lgb.Dataset(X_train, y_train)
        self.lgb_eval = lgb.Dataset(X_test, y_valid, reference=self.lgb_train)
    
    def set_test_data(self, test):
        self.test = test
    
    def set_categorical(self, categorical:list):
        self.categorical_features = categorical
        
    def fit(self):
        self.gbm = lgb.cv(self.params, 
                          self.lgb_train, 
                          valid_sets=self.lgb_eval, 
                          num_boost_round=self.num_boost_round,
                          categorical_feature=self.categorical_features,
                          callbacks=self.callback)
        
        return self.gbm
        
    def get_oof(self):
        return self.gbm.predict_proba(self.X_valid, num_iteration=gbm.self.best_iteration)[:, 1]
        
    def get_sub(self):
        return self.gbm.predict_proba(self.test, num_iteration=gbm.self.best_iteration)[:, 1]
    
    def get_importance(self):
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = self.X_train.columns
        fold_importance_df["importance_gain"] = self.gbm.feature_importance() #self.clf.booster_.feature_importance(importance_type='gain')
        fold_importance_df.sort_values("importance_gain", ascending=False, inplace=True)
        return fold_importance_df

